# Detecting Building defects using  VGG16

### Importing necessary libraries

In [1]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob


### Image Data Agumentation¶

In [2]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#performing data agumentation on train data
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#performing data agumentation on test data
test_datagen = ImageDataGenerator(rescale = 1./255)

### Loading our data and performing data agumentation

In [3]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(r'C:\\Users\prash\Desktop\VEC Modified projects\Wild_ediblity\Wild_ediblity\dataset\training_set',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 624 images belonging to 8 classes.


In [4]:
test_set = test_datagen.flow_from_directory(r'C:\\Users\prash\Desktop\VEC Modified projects\Wild_ediblity\Wild_ediblity\dataset\test_set',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 269 images belonging to 8 classes.


In [5]:
print(training_set.class_indices)#checking the number of classes

{'Mountain Laurel_nonedible': 0, 'Peppergrass_edible': 1, 'Purple Deadnettle_edible': 2, 'Rhododendron_nonedible': 3, 'Toothwort_edible': 4, 'Wild Grape Vine_edible': 5, 'Wild Leek_edible': 6, 'rattlebox_nonedible': 7}


In [6]:
from collections import Counter as c
c(training_set .labels)

Counter({0: 72, 1: 120, 2: 72, 3: 72, 4: 72, 5: 72, 6: 72, 7: 72})

## Model Building

In [7]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = r'C:\\Users\prash\Desktop\VEC Modified projects\Wild_ediblity\Wild_ediblity\dataset\training_set'
valid_path = r'C:\\Users\prash\Desktop\VEC Modified projects\Wild_ediblity\Wild_ediblity\dataset\test_set'


In [8]:
# Import the Vgg 16 library as shown below and add preprocessing layer to the front of VGG
# Here we will be using imagenet weights

vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [9]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [10]:
  # useful for getting number of output classes
folders = glob(r'C:\\Users\prash\Desktop\VEC Modified projects\Wild_ediblity\Wild_ediblity\dataset\training_set\*')

In [11]:
folders

['C:\\\\Users\\prash\\Desktop\\VEC Modified projects\\Wild_ediblity\\Wild_ediblity\\dataset\\training_set\\Mountain Laurel_nonedible',
 'C:\\\\Users\\prash\\Desktop\\VEC Modified projects\\Wild_ediblity\\Wild_ediblity\\dataset\\training_set\\Peppergrass_edible',
 'C:\\\\Users\\prash\\Desktop\\VEC Modified projects\\Wild_ediblity\\Wild_ediblity\\dataset\\training_set\\Purple Deadnettle_edible',
 'C:\\\\Users\\prash\\Desktop\\VEC Modified projects\\Wild_ediblity\\Wild_ediblity\\dataset\\training_set\\rattlebox_nonedible',
 'C:\\\\Users\\prash\\Desktop\\VEC Modified projects\\Wild_ediblity\\Wild_ediblity\\dataset\\training_set\\Rhododendron_nonedible',
 'C:\\\\Users\\prash\\Desktop\\VEC Modified projects\\Wild_ediblity\\Wild_ediblity\\dataset\\training_set\\Toothwort_edible',
 'C:\\\\Users\\prash\\Desktop\\VEC Modified projects\\Wild_ediblity\\Wild_ediblity\\dataset\\training_set\\Wild Grape Vine_edible',
 'C:\\\\Users\\prash\\Desktop\\VEC Modified projects\\Wild_ediblity\\Wild_ediblity\\

In [12]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [13]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [14]:

# view the structure of the model
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Compiling the model

In [15]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


### Fit the model

In [16]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\prash\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
 6/20 [========>.....................] - ETA: 2:45 - loss: 2.8379 - accuracy: 0.1364

KeyboardInterrupt: 

In [ ]:
# plot the loss
import matplotlib.pyplot as plt
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

### Saving our model

In [ ]:
# save it as a h5 file

from tensorflow.keras.models import load_model

model.save('WildPlantEdibility.h5')

### Predicitng our results

In [ ]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("WildPlantEdibility.h5") #loading the model for testing

In [ ]:
img=image.load_img(r'C:\\Users\prash\Desktop\VEC Modified projects\Wild_ediblity\Wild_ediblity\dataset\test_set\rattlebox_nonedible\images116.jpg',target_size=(224,224))
x=image.img_to_array(img)
#x=x/255C:\Users\HP\Desktop\Wild_ediblity\dataset\training_set
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
#model.predict(img_data)
a=np.argmax(model.predict(img_data), axis=1)


In [ ]:
a

In [ ]:
index=['Mountain Laurel_nonedible', 'Peppergrass_edible', 'Purple Deadnettle_edible', 'Rhododendron_nonedible', 'Toothwort_edible', 'Wild Grape Vine_edible', 'Wild Leek_edible', 'rattlebox_nonedible']
result=str(index[a[0]])
result